In [270]:
import numpy as np
import pandas as pd

### Search recall

In [291]:
df = pd.read_csv('../data/results/eval_ek.csv')

In [292]:
df.shape

(338, 13)

In [293]:
df.shape

(338, 13)

In [294]:
df.groupby(['town', 'district']).ngroups

107

In [295]:
sdf = df.groupby(by=['town', 'district']).agg({"correct_page_searched": "sum"}).reset_index()

In [296]:
sdf["correct_page_searched"].value_counts()

1    94
0     9
2     4
Name: correct_page_searched, dtype: int64

In [297]:
sdf["correct"] = sdf["correct_page_searched"].apply(lambda x: 1 if x > 0 else 0)

In [298]:
recall = sdf["correct"].sum() / sdf.shape[0]
print("recall =", recall)

recall = 0.9158878504672897


In [299]:
incorrect_search = sdf[sdf["correct"] == 0]["town"].values.tolist()

In [300]:
df[df["town"].isin(incorrect_search)]

,town,district,expected,actual,correct_page_searched,correct_page_extracted,gt_page,searched_pages,searched_pages_expanded,extracted_pages,actual_normalized,expected_normalized,correct_answer
30,clinton,Business-2,10000,NaN,False,False,{334},{18},"{17, 18}",set(),NaN,10000.0,False
31,clinton,Business-2,10000,NaN,False,False,{334},{303},"{304, 305, 302, 303}",set(),NaN,10000.0,False
32,clinton,Business-2,10000,NaN,False,False,{334},{348},"{347, 348}",set(),NaN,10000.0,False
33,clinton,Business-2,10000,NaN,True,False,{334},{334},"{333, 334}",set(),NaN,10000.0,False
34,clinton,Business-2,10000,NaN,False,False,{334},{456},"{456, 457, 455}",set(),NaN,10000.0,False
35,clinton,Business-2,10000,NaN,False,False,{334},{446},"{445, 446, 447}",set(),NaN,10000.0,False
36,clinton,Village,10000,NaN,False,False,{334},{325},"{324, 325, 326, 327, 328, 329}",set(),NaN,10000.0,False
37,clinton,Village,10000,NaN,False,False,{334},{127},"{128, 126, 127}",set(),NaN,10000.0,False
38,clinton,East Main Street Village,10000,NaN,False,False,{334},{18},"{17, 18}",set(),NaN,10000.0,False
39,clinton,East Main Street Village,10000,NaN,False,False,{334},{348},"{347, 348}",set(),NaN,10000.0,False


### Answer accuracy

In [182]:
df_old = pd.read_csv('../data/results/eval.csv')

In [228]:
adf = df_old.groupby(by=['town', 'district']).agg({"correct_answer": "sum"}).reset_index()

In [229]:
adf["correct_answer"].value_counts()

1    53
0    39
2     7
Name: correct_answer, dtype: int64

In [230]:
adf["correct"] = adf["correct_answer"].apply(lambda x: 1 if x > 0 else 0)
acc = adf["correct"].sum() / adf.shape[0]
print("accuracy =", acc)

accuracy = 0.6060606060606061


In [233]:
incorrect_answer = adf[adf["correct"] == 0]["town"].values.tolist()

In [235]:
df_inc = df_old[df_old["town"].isin(incorrect_answer)]

In [238]:
df_inc[df_inc["correct_page_searched"]]

,town,district,expected,actual,correct_page_searched,correct_page_extracted,gt_page,searched_pages,extracted_pages,actual_normalized,expected_normalized,correct_answer
3,beacon-falls,Planned Adaptive Reuse Development 3,6229080,35 Feet,True,True,{121},{121},"{120, 121, 122}",NaN,6229080.0,False
50,ellington,Designed Multifamily,"217800, 435600",50 ft,True,False,{23},{23},{24},NaN,217800.0,False
51,ellington,Designed Multifamily,"217800, 435600",50 ft,True,False,{23},{23},{24},NaN,435600.0,False
78,ledyard,Commercial Marine,40000,40k,True,False,{16},{16},{15},NaN,40000.0,False
83,litchfield,Gateway Residences,40000,"30,000 sq ft",True,True,{11},{11},"{11, 12, 13}",30000.0,40000.0,False
111,newtown,Business & Professional Office,43560,1 acre,True,False,{123},{123},{122},43560.0,43560.0,True
113,newtown,Farming & Residential-R-2,87120,0.5 acres,True,False,{123},{123},{122},21780.0,87120.0,False
119,norfolk,Village Business,5000,"5,000 SF",True,False,{45},{45},{44},NaN,5000.0,False
170,suffield,R-90 Single Family Residential,90000,Ten (10) acres,True,True,"{41, 47}",{47},"{48, 46, 47}",435600.0,90000.0,False
200,watertown,Residential R-20,"20000,40000",40000 sq ft,True,False,{89},{89},{88},40000.0,20000.0,False
